In [2]:
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib import response
import time
import pandas as pd
import json

In [3]:
# driver 오류
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [5]:
place_url_list = pd.read_csv("../data/tripcom_place_url_list.csv")
place_url_list = place_url_list["URL"]

In [38]:
place_url_list[20]

'https://kr.trip.com/travel-guide/attraction/gangneung-si/ojukheon-house-96184/'

In [30]:
len(place_url_list)

682

In [44]:
# URL 접속
with open('../Gangwon_trip_project/config.json', 'r') as f:
    config = json.load(f)

driver = webdriver.Chrome(config["Chromedriver"],options=options)
driver.get("https://google.com")

# 데이터 저장
score_list = []                # 리뷰 평점 
score_date_list = []       # 리뷰 작성일
coment_list = []            # 리뷰(원문)
coment_trans_list = []  # 리뷰(번역)
place_name = []             # 리뷰 장소



# 페이지 들어가기
start_place_number = 1
end_place_number = 2
place_number = start_place_number
for place in place_url_list[start_place_number:end_place_number]:
    review_page = 0
    place_number += 1
    driver.get(place)
    # time.sleep(1)
    time.sleep(3)

    # 장소
    place_css = driver.find_element_by_css_selector("h1.basicName")

    # 리뷰
    review_pages = driver.find_elements_by_css_selector("div.gl-poi-detail_page ul.gl-cpt-pager > li")
    try :
        review_page = review_pages[-1].text
    except IndexError as e:
        print(review_pages)

    print(f'총 페이지 수 : {review_page}')
    try: 
        for page in range(int(review_page)):
            print(f'{place_number}번장소 : {place_css.text}')
            print(f'페이지 : {page+1}/{review_page} 실행 중')
            time.sleep(3)
            try:
                score_css = driver.find_elements_by_css_selector("div.gl-poi-detail_comment-content > div.ovh > span.review_score.score-name")
                for score in score_css:
                    score_list.append(score.text)
            except:
                print("e")
                

                    
            date_css = driver.find_elements_by_css_selector("span.r.c2.create-time")
            coment_css = driver.find_elements_by_css_selector('div.gl-poi-detail_comment-content > div > a')
            coment_trans_css = driver.find_elements_by_css_selector('div.gl-poi-detail_comment-content > div > a > p')
            
            

            for point in date_css:
                date = point.text
                date = date.split(" ")
                date = date[1] + date[2] + date[3]
                score_date_list.append(date)


            ## 리뷰(원문)
            for place in coment_css :
                coment = place.get_attribute("alt")
                coment_list.append(coment)


            ## 리뷰(번역)
            for place in coment_trans_css:
                coment_trans = place.text
                coment_trans_list.append(coment_trans)
                place_name.append(place_css.text)
            
            if page < (int(review_page)-1):
                driver.find_element_by_css_selector('div.gl-poi-detail_page div.gl-cpt-pagination > button.btn-next').send_keys(Keys.ENTER)
            print(f'페이지 {page+1}/{review_page} 실행 완료')
    except :
        pass


총 페이지 수 : 3
2번장소 : 강원도립화목원
페이지 : 1/3 실행 중
페이지 1/3 실행 완료
2번장소 : 강원도립화목원
페이지 : 2/3 실행 중
페이지 2/3 실행 완료
2번장소 : 강원도립화목원
페이지 : 3/3 실행 중
페이지 3/3 실행 완료


In [46]:
review_table = []
review_table = [place_name,score_list,score_date_list,coment_list,coment_trans_list]
review_table = pd.DataFrame(review_table).T
review_table.columns = ["장소","평점","작성일","리뷰(원문)","리뷰(번역)"]
review_table["평점"] = review_table["평점"].replace(["완벽해요!","최고에요!","좋아요!","보통이에요","최악이에요"],[5,4,3,2,1])

pd.set_option("display.max_rows",200)
review_table

,장소,평점,작성일,리뷰(원문),리뷰(번역)
0,강원도립화목원,4,2017년1월3일,江原道立花木园位于湖畔之城春川的北部，1996.2至1998.10间，司农洞示范地上设立起乡...,"강원도 립 화원은 호숫가 도시인 춘천 북부에 위치하고 있으며, 1996 년 2 월부..."
1,강원도립화목원,5,2016년12월18일,春川 南怡岛，曾是多部电视剧的拍摄地。特别是电视剧&ldquo;冬季恋歌&rdquo;的摄影...,춘천 남이섬은 한때 많은 드라마 촬영지였습니다. 특히 TV 시리즈 '겨울 연가'촬영...
2,강원도립화목원,5,2015년9월12일,很美丽的园子，池塘，花草树木都很美，还有个山林博物馆，可以玩半天。,정원은 매우 아름답고 연못과 꽃과 나무가 아름답고 산 박물관도있어 반나절 동안 놀 ...
3,강원도립화목원,5,2015년10월10일,这个季节去江原道，要多带一件外套，山上面会冷，有点寒。。,이번 시즌 강원도에 가면 여분의 털을 챙겨 가야하는데 산은 춥고 조금 추울 것이다. .
4,강원도립화목원,4,2014년12월9일,位于湖畔之城春川的北部，四季景色各有魅力，只有身临其境才能欣赏到山水如画的美。有乡土花草展馆...,호숫가 도시인 춘천 북부에 위치한 사계절의 풍경은 그 자체의 매력이 있으며 현장에있...
5,강원도립화목원,4,2015년9월5일,江原道立花木园位于湖畔之城春川的北部，如今的立花木园既是学生的自然课堂，又是市民们喜爱的休憩场所。,"강원도 타치바나 정원은 호숫가 도시인 춘천 북부에 위치해 있으며, 오늘날 타치바나 ..."
6,강원도립화목원,5,2016년1월25일,江原道立花木园，有树木园、草坪、荷花池、喷泉等，四季景色都很美，是市民悠闲的场所。,"강원도 스탠딩 플라워 가든에는 수목원, 잔디밭, 연꽃 연못, 분수 등이 있습니다. ..."
7,강원도립화목원,5,2015년9월19일,花木园是一个非常好的植物园，这里有湖水和池塘，景色非常优美的。,Huamu Garden은 호수와 연못이있는 매우 좋은 식물원이며 풍경이 매우 아름답...
8,강원도립화목원,4,2015년9월4일,这里是最浪漫的韩剧取景地，千万不要错过，拍照很美的。,이것은 한국 드라마의 가장 로맨틱 한 장소입니다. 놓치지 마세요. 사진도 아름답습니다.
9,강원도립화목원,5,2017년7월12일,地方很大，有很多花草树木，景色优美，适合拍照休闲,장소는 매우 크고 꽃과 나무가 많으며 경치가 아름답고 사진 촬영 및 레저에 적합합니다.
